In [1]:
import cv2
import os
from mtcnn import MTCNN
from IPython.display import display
from PIL import Image
from deepface import DeepFace
import numpy as np
import joblib
detector = MTCNN()

In [4]:
def save_cropped_images(dataset_path):
    dataset_parent_directory_path = os.path.dirname(dataset_path)
    cropped_dataset_path = os.path.join(dataset_parent_directory_path,'cropped_faces_dataset')
    if not os.path.exists(cropped_dataset_path):
        os.makedirs(cropped_dataset_path)

    for rollnumber in os.listdir(dataset_path):
        rollnumber_path = os.path.join(dataset_path,rollnumber)
        new_rollnumber_path = os.path.join(cropped_dataset_path,rollnumber)
        if not os.path.exists(new_rollnumber_path):
            os.makedirs(new_rollnumber_path)
        print(rollnumber)
        for file_name in os.listdir(rollnumber_path):
            each_file_path = os.path.join(rollnumber_path,file_name)
            each_image = cv2.imread(each_file_path)
            faces = detector.detect_faces(each_image)
            if len(faces)==0:
                continue
            face = faces[0]
            x,y,w,h = face['box']
            face_image = each_image[y:y+h,x:x+w]
            image_saving_path = os.path.join(new_rollnumber_path,file_name)
            cv2.imwrite(image_saving_path,face_image)


In [5]:
#save_cropped_images(r'C:\Users\Venkatesh Yeturi\OneDrive\Desktop\auto_attendance_python\Data\dataset')

224102301
1/1 [==============================] - 0s 68ms/step
224102302
1/1 [==============================] - 0s 42ms/step
224102303
1/1 [==============================] - 0s 40ms/step
224102304
1/1 [==============================] - 0s 28ms/step
224102305
1/1 [==============================] - 0s 31ms/step
224102306
1/1 [==============================] - 0s 46ms/step
224102307
1/1 [==============================] - 0s 40ms/step
224102308
1/1 [==============================] - 0s 43ms/step
224102309
1/1 [==============================] - 0s 45ms/step
224102310
1/1 [==============================] - 0s 55ms/step
224102312
1/1 [==============================] - 0s 46ms/step
224102314
1/1 [==============================] - 0s 52ms/step
224102315
1/1 [==============================] - 0s 73ms/step
224102317
1/1 [==============================] - 0s 70ms/step
224102319
1/1 [==============================] - 0s 44ms/step
224102320
1/1 [==============================] - 0s 51ms/step
22410232

In [2]:

def get_embeddings(roll_path):
    embeddings = []
    labels = []
    for file in os.listdir(roll_path):
        file_path = os.path.join(roll_path,file)
        embedding = DeepFace.represent(file_path, model_name='Facenet',enforce_detection=False)
        embeddings.append(embedding[0]['embedding'])
    roll_number = os.path.basename(roll_path)
    labels = [roll_number]*len(os.listdir(roll_path))  
    return embeddings,labels

data_path = r'C:\Users\Venkatesh Yeturi\OneDrive\Desktop\auto_attendance_python\Data\cropped_faces_dataset'

embeddings = []
labels = []
for each_roll in os.listdir(data_path):
    roll_path = os.path.join(data_path,each_roll)
    person_embeddings, person_labels = get_embeddings(roll_path)
    embeddings.append(person_embeddings)
    labels.append(person_labels)
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(embeddings, labels)
joblib.dump(knn,'knn_model.joblib')

def get_predicted_roll_numbers(test_image_path):
    test_image = cv2.imread(test_image_path)
    faces = detector.detect_faces(test_image)
    parent_folder_path = os.path.dirname(test_image_path)
    str = os.path.basename(test_image_path) + "_faces"
    new_faces_folder = os.path.join(parent_folder_path,str)
    if not os.path.exists(new_faces_folder):
        os.makedirs(new_faces_folder)
    roll_numbers = []
    for i,face in enumerate(faces):
        x,y,w,h = face['box']
        face_image = test_image[y:y+h,x:x+w]
        image_saving_path = os.path.join(new_faces_folder,f'face_{i}.jpg')
        cv2.imwrite(image_saving_path,face_image)
        face_embedding = DeepFace.represent(image_saving_path,model_name='Facenet',enforce_detection=False)[0]['embedding']
        roll_number = knn.predict([face_embedding])
        roll_numbers.append(roll_number)
    return roll_numbers



In [ ]:
###########################################

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(embeddings, labels)

KNeighborsClassifier(n_neighbors=3)

In [18]:
def get_predicted_roll_numbers(test_image_path):
    test_image = cv2.imread(test_image_path)
    faces = detector.detect_faces(test_image)
    parent_folder_path = os.path.dirname(test_image_path)
    str = os.path.basename(test_image_path) + "_faces"
    new_faces_folder = os.path.join(parent_folder_path,str)
    if not os.path.exists(new_faces_folder):
        os.makedirs(new_faces_folder)
    roll_numbers = []
    for i,face in enumerate(faces):
        x,y,w,h = face['box']
        face_image = test_image[y:y+h,x:x+w]
        image_saving_path = os.path.join(new_faces_folder,f'face_{i}.jpg')
        cv2.imwrite(image_saving_path,face_image)
        face_embedding = DeepFace.represent(image_saving_path,model_name='Facenet',enforce_detection=False)[0]['embedding']
        roll_number = knn.predict([face_embedding])
        roll_numbers.append(roll_number)
    return roll_numbers

In [19]:
test_image_1 = r"C:\Users\Venkatesh Yeturi\OneDrive\Desktop\auto_attendance_python\Data\test_images\venky-ritesh-arpit.jpg"
for roll in get_predicted_roll_numbers(test_image_1):
    print(roll)

1/1 [==============================] - 0s 48ms/step
['224102302']
['224102322']
